<a href="https://colab.research.google.com/github/ASHIKalip/DataScience/blob/main/openI_VQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this notebook, we will classify 15 thoracic findings from Chest X-ray images and associated reports. This can be considered as an VQA task. We will fine-tune 3 pre-trained transformer based V+L models. After running through this notebook, you will be able to fine-tune these models on your customized dataset.**

####**0.1 clone our repo and install dependencies!**


In [1]:
!git clone https://github.com/YIKUAN8/Transformers-VQA.git
%cd Transformers-VQA/
!pip install -r requirements.txt


Cloning into 'Transformers-VQA'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 182 (delta 64), reused 180 (delta 63), pack-reused 0 (from 0)
Receiving objects: 100% (182/182), 1.94 MiB | 4.91 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/Transformers-VQA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.7 MB/s eta 0:00:00


**Change the 79th line of param.py from**
```
args = parser.parse_args()
```
to
```
args = parser.parse_args([])
```
This will enable us to use *argparse* in jupyter notebook!



####**0.2 Download pre-trained models and place them to data/pretrained/, you could choose from [VisualBERT](https://github.com/uclanlp/visualbert), [LXMERT](https://github.com/airsplay/lxmert), [UNITER](https://github.com/ChenRocks/UNITER).**

In [16]:
#line 1: UNITER; line 2:LXMERT, line 3: VisualBERT. Comment out selected lines if you don't want to use this model
#if the pre-trained VisualBERT cannot be downloaded succesfully, rerun one more time or refer to this link: https://drive.google.com/file/d/1kuPr187zWxSJbtCbVW87XzInXltM-i9Y/view?usp=sharing
#!wget https://convaisharables.blob.core.windows.net/uniter/pretrained/uniter-base.pt -P models/pretrained/
#!wget --no-check-certificate https://nlp1.cs.unc.edu/data/model_LXRT.pth -P models/pretrained/
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y" -O models/pretrained/visualbert.th && rm -rf /tmp/cookies.txt


--2024-11-14 15:25:37--  https://docs.google.com/uc?export=download&confirm=&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y
Resolving docs.google.com (docs.google.com)... 108.177.11.138, 108.177.11.102, 108.177.11.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.11.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y&export=download [following]
--2024-11-14 15:25:37--  https://drive.usercontent.google.com/download?id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.196.132, 2607:f8b0:400c:c36::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.196.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&continue=https://drive.usercontent.google.com/download?i

####**0.3 Download OpenI dataset.**

A detailed description of this dataset can be found at [here](https://openi.nlm.nih.gov/). In summary, there are 3684 CXR Image-Report pairs in this dataset. Each pair has an annotation of 15 throacic findings from MESH terms. We convert the raw data to a dataframe with better visibility. It can be accessed with the following command or this [link](https://drive.google.com/file/d/1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG/view?usp=sharing).

In [17]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG" -O data/openIdf.csv && rm -rf /tmp/cookies.txt


--2024-11-14 15:26:10--  https://docs.google.com/uc?export=download&confirm=&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG
Resolving docs.google.com (docs.google.com)... 108.177.11.138, 108.177.11.100, 108.177.11.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.11.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG&export=download [following]
--2024-11-14 15:26:10--  https://drive.usercontent.google.com/download?id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.196.132, 2607:f8b0:400c:c36::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.196.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1138211 (1.1M) [application/octet-stream]
Saving to: ‘data/openIdf.csv’

data/openIdf.csv    100%[===================>]   1.08M  --

***0.3.1 Have a glance of this dataframe, column 'TXT' is the radiology report; column 'split' and 'id' are self-explantory; All other columns are the 15 findings. Our task will be a 15-labels binary classification with visual and semantic input.***

In [18]:
import pandas as pd
openI = pd.read_csv('data/openIdf.csv',index_col=0)
openI.head()

,id,Atelectasis,Cardiomegaly,Effusion,Infiltration,Mass,Nodule,Pneumonia,Pneumothorax,Consolidation,Edema,Emphysema,Fibrosis,Pleural_Thickening,Hernia,Normal,split,TXT
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,The cardiac silhouette and mediastinum size ar...
1021,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,train,Borderline cardiomegaly. Midline sternotomy XX...
2020,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,"No displaced rib fractures, pneumothorax, or ..."
3061,4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,train,There are diffuse bilateral interstitial and a...
3169,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,train,The cardiomediastinal silhouette and pulmonary...


####**0.4 Download the visaul features extracted by BUTD. 36 2048-dimension visual feature is extracted from each CXR Image. We use this [implementation](https://github.com/airsplay/py-bottom-up-attention). This step will take a while (~1min). To save downloading time, you can also make a copy of this [shareable link](https://drive.google.com/file/d/1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ/view?usp=sharing) to your own google drive and mount you colab to your gdrive.**


*If you are interested in the original CXR images, which is unnecessary to out project , you can access them [here](https://drive.google.com/drive/folders/1s5A0CFB6-2N5ThbuorUK1t-bUEKmZnjz?usp=sharing).*

In [19]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ" -O data/openI_v_features.pickle && rm -rf /tmp/cookies.txt

--2024-11-14 15:26:20--  https://docs.google.com/uc?export=download&confirm=&id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ
Resolving docs.google.com (docs.google.com)... 108.177.11.138, 108.177.11.100, 108.177.11.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.11.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ&export=download [following]
--2024-11-14 15:26:20--  https://drive.usercontent.google.com/download?id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.196.132, 2607:f8b0:400c:c36::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.196.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2440 (2.4K) [text/html]
Saving to: ‘data/openI_v_features.pickle’

data/openI_v_featur 100%[===================>]   2.38K  --.-KB/s

***0.4.1 Load visual features***

In [21]:
# Install gdown if not already installed
!pip install gdown

# Use gdown to download the file
import gdown
file_id = '1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ'
gdown.download(f"https://drive.google.com/uc?id={file_id}", "openI_visual_features.pickle", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ
From (redirected): https://drive.google.com/uc?id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ&confirm=t&uuid=7295f314-9409-490a-a392-348749d94a01
To: /content/Transformers-VQA/openI_visual_features.pickle
100%|██████████| 1.09G/1.09G [00:15<00:00, 70.8MB/s]


'openI_visual_features.pickle'

In [24]:
import pickle
openI_v_f = pickle.load( open( "/content/Transformers-VQA/openI_visual_features.pickle", "rb" ) )

In [25]:
assert set(list(openI_v_f.keys())) == set(openI.id.values), "Visual Features are inconsistent with openI dataset"

In [26]:
feature_example, bbox_example, (img_w_example, img_h_example) = openI_v_f[openI.id.iloc[0]]

In [27]:
feature_example.shape, bbox_example.shape, (img_w_example, img_h_example)

((36, 4), (36, 2048), (420, 512))

####**Now We have download all data, models, and dependencies. We are good to go!!!**
**1. Change default arguments**

First, let's check it out!

In [28]:
from param import args

In [29]:
args.__dict__

{'model': 'visualbert',
 'train': 'train,nominival',
 'valid': 'minival',
 'test': None,
 'batch_size': 18,
 'optim': 'bert',
 'lr': 0.0001,
 'epochs': 2,
 'dropout': 0.1,
 'seed': 9595,
 'max_seq_length': 128,
 'output': 'models/trained/',
 'fast': False,
 'tiny': False,
 'tqdm': True,
 'load_trained': None,
 'load_pretrained': '/content/Transformers-VQA/models/pretrained/visualbert.th',
 'from_scratch': False,
 'mce_loss': False,
 'multiGPU': False,
 'num_workers': 0,
 'optimizer': 'bert'}

***1.1*** Let's overwrite some arguments***

In [30]:
args.batch_size = 18
args.epochs = 2
args.model = 'visualbert' # use visualbert
args.load_pretrained = '/content/Transformers-VQA/models/pretrained/visualbert.th' #load pretrained visualbert model
args.max_seq_length = 128 #truncate or pad report lengths to 128 subwords

####**2. Create customized dataloader**

In [31]:
findings = list(openI.columns[1:-2])
findings

['Atelectasis',
 'Cardiomegaly',
 'Effusion',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pneumonia',
 'Pneumothorax',
 'Consolidation',
 'Edema',
 'Emphysema',
 'Fibrosis',
 'Pleural_Thickening',
 'Hernia',
 'Normal']

In [32]:
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
class OpenIDataset(Dataset):
  def __init__(self, df, vf, split, model = 'lxmert'):
    # train_test_split and prepare labels
    self.dataset = df[df['split'] == split]
    self.visual_features = vf
    self.id_list = self.dataset.id.tolist()
    self.report_list = self.dataset.TXT.tolist()
    self.findings_list = self.dataset.columns[1:-2]
    self.target_list = self.dataset[self.findings_list].to_numpy().astype(np.float32)
    self.model = model

  def __len__(self):
    return len(self.id_list)

  def __getitem__(self, item):
    cxr_id = self.id_list[item]
    target = self.target_list[item]
    boxes, feats, (img_w, img_h) = self.visual_features[cxr_id]
    report = self.report_list[item]
    if self.model == 'uniter':
      boxes = self._uniterBoxes(boxes)
    if self.model == 'lxmert':
      boxes[:, (0, 2)] /= img_w
      boxes[:, (1, 3)] /= img_h
    return cxr_id, feats, boxes, report, target

  def _uniterBoxes(self, boxes):#uniter requires a 7-dimensiom beside the regular 4-d bbox
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes[:,1] = boxes[:,0]
    new_boxes[:,0] = boxes[:,1]
    new_boxes[:,3] = boxes[:,2]
    new_boxes[:,2] = boxes[:,3]
    new_boxes[:,4] = new_boxes[:,3]-new_boxes[:,1] #w
    new_boxes[:,5] = new_boxes[:,2]-new_boxes[:,0] #h
    new_boxes[:,6]=new_boxes[:,4]*new_boxes[:,5] #area
    return new_boxes

In [33]:
training = OpenIDataset(df = openI, vf = openI_v_f,  split='train', model = args.model)
testing = OpenIDataset(df = openI, vf = openI_v_f,  split='test', model = args.model)

In [34]:
train_loader = DataLoader(training, batch_size=args.batch_size,shuffle=True, num_workers=0,drop_last=True, pin_memory=True)
test_loader = DataLoader(testing, batch_size=128,shuffle=False, num_workers=0,drop_last=False, pin_memory=True)

####**3. Model, Optimizer, Loss Function, and Evaluation Function**

In [35]:
from vqa_model import VQAModel
#init model
model = VQAModel(num_answers = len(findings), model = args.model)

100%|██████████| 407873900/407873900 [00:10<00:00, 39409709.87B/s]
/content/Transformers-VQA/src/modeling.py:780: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = 

In [36]:
#load pretrained weights
model.encoder.load(args.load_pretrained)

/content/Transformers-VQA/src/entry.py:196: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path)


UnpicklingError: invalid load key, '<'.

In [37]:
#send to GPU
model = model.cuda()


In [38]:
import torch
loss = torch.nn.BCEWithLogitsLoss()

In [39]:
from src.optimization import BertAdam
optim = BertAdam(list(model.parameters()),lr=args.lr,warmup=0.1,t_total=len(train_loader)*args.epochs)
# t_total denotes total training steps
# batch_per_epoch = len(train_loader)
# t_total = int(batch_per_epoch * args.epochs)

In [40]:
# Evaluation function, we will report the AUC and accuray of each finding
def eval(target, pred):
    acc_list = []
    for i, d in enumerate(findings[:-1]): #normal is excluded
        acc = np.mean(target[:,i] == (pred[:,i]>=0.5))
        print(i,d,acc)
        acc_list.append(acc)
    print('Averaged: '+str(np.average(acc_list)))

In [41]:
sgmd = torch.nn.Sigmoid()

####**4. HIT and RUN**

In [42]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(sgmd(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/161 [00:00<?, ?it/s]

/content/Transformers-VQA/src/optimization.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1642.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Epoch 0: Training Loss: tensor(2.2234, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/7 [00:00<?, ?it/s]

0 Atelectasis 0.8989637305699482
1 Cardiomegaly 0.8963730569948186
2 Effusion 0.9455958549222798
3 Infiltration 0.9715025906735751
4 Mass 0.9961139896373057
5 Nodule 0.966321243523316
6 Pneumonia 0.9857512953367875
7 Pneumothorax 0.9935233160621761
8 Consolidation 0.9857512953367875
9 Edema 0.9805699481865285
10 Emphysema 0.9676165803108808
11 Fibrosis 0.9935233160621761
12 Pleural_Thickening 0.9792746113989638
13 Hernia 1.0
Averaged: 0.9686343449296818


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(1.8493, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/7 [00:00<?, ?it/s]

0 Atelectasis 0.8989637305699482
1 Cardiomegaly 0.8963730569948186
2 Effusion 0.9455958549222798
3 Infiltration 0.9715025906735751
4 Mass 0.9961139896373057
5 Nodule 0.966321243523316
6 Pneumonia 0.9857512953367875
7 Pneumothorax 0.9935233160621761
8 Consolidation 0.9857512953367875
9 Edema 0.9805699481865285
10 Emphysema 0.9676165803108808
11 Fibrosis 0.9935233160621761
12 Pleural_Thickening 0.9792746113989638
13 Hernia 1.0
Averaged: 0.9686343449296818
